# Comunitat Valenciana' pharmacies' import

This jupyter notebook contains the script for importing the pharmacies in Comunitat Valenciana into OSM as well as the documentation of the whole process in a single file, making it easier to review both the process and the results as well as the decisions taken.

The goal is to manually merge and import all the pharmacies' information provided by Generalitat Valenciana, while testing the scripts for data preparation.

## Data Sources

* http://www.san.gva.es/web/dgfps/relacion-de-oficinas-de-farmacia

## License

We have requested autorization due to COV19 emergency. Data is released under CC

## Import type

This import will be done manually, using JOSM to edit the data. Consider using Task Manager.

## Data preparations

All data preparations will be made automatically in this notebook.



In [9]:
import pandas as pd
import geopandas as gpd
import geopy
from osmi_helpers import data_gathering as osmi_dg

# Define Data Sources

DATA_RAW = 'data/raw/ListadoOficinasFarmacia.csv'

CSV_PARSER = 'fields_mapping.csv'

### Fields' mapping.

In [10]:
# Read CSV file with fields' mapping and description.
fields_mapping = pd.read_csv(CSV_PARSER)

# Display table.
fields_mapping



,Original field,Description,OSM tagging,Comments
0,ESTABLECIMIENTO,Internal ID,source:pkey,NaN
1,PROVINCIA,Province,addr:province,NaN
2,MUNICIPIO,Municipality,addr:city,NaN
3,TITULAR,NaN,name,NaN
4,DIRECCIÓN,Address,addr:full,"Full address, to be splitted into `adr:street`..."
5,DEPARTAMENTO DE SALUD,NaN,NaN,Not imported
6,ZONA ZBS,NaN,NaN,Not imported
7,ZONA FARMACÉUTICA,NaN,NaN,Not imported


## Data gathering

Run the code below to download original datasources and convert them into a dataframe.

In [11]:
# Download a file and convert it into a dataframe.
df_raw = pd.read_csv(DATA_RAW)

df_raw.head(10)


,ESTABLECIMIENTO,PROVINCIA,MUNICIPIO,TITULAR,DIRECCIÓN,DEPARTAMENTO DE SALUD,ZONA ZBS,ZONA FARMACÉUTICA
0,A-164-F,ALICANTE,AGOST,CARLOS GISBERT ARQUES,"PLAZA DE ESPAÑA, 19, 03698",19,8,25
1,A-285-F,ALICANTE,AGOST,FRANCISCO AYUSO MACIA,"AVDA Virgen de la Paz, 30, 03698",19,8,25
2,A-510-F,ALICANTE,AGRES,INMACULADA FERRERO PEREZ,"CALLE San Antonio, 13, 03837",15,9,15
3,A-540-F,ALICANTE,AIGUES,JAVIER VILLAMAYOR PIÑAS,"CALLE CANALEJAS, 12, 03569",17,6,22
4,A-13-F,ALICANTE,ALBATERA,ISABEL BALSAMEDA MORALES,"CALLE Ramon y Cajal, 6, 03340",21,1,35
5,A-14-F,ALICANTE,ALBATERA,"MARIA JOSE BALMASEDA DEL ALAMO, ARTURO CORBACH...","CTRA HONDON. EDIF.ROMEO, S/N, 03340",21,1,35
6,A-580-F,ALICANTE,ALBATERA,MARIA DEL MAR GARCIA MOLINA,"AVDA Calvario, 39, 03340",21,1,35
7,A-814-F,ALICANTE,ALBATERA,FRANCISCO GARCIA CANOVAS,"CALLE Meson, 45, 03340",21,1,35
8,A-531-F,ALICANTE,ALCALALI,ENCARNACION MOLL MENGUAL,"C Calvari, 7, 03728",13,1,1
9,A-748-F,ALICANTE,ALCALALI,RAQUEL FERRER GONZALEZ,"CALLE SALAMANCA, 19, 03723",13,8,1


## Data conversion

Run the cell below to convert raw data into a suitable OSM-friendly structure, according to the provided CSV fields with fields' mappings stated in `CSV_PARSER` variable.

In [12]:
# Selects and renames fields according to CSV parser.
df = osmi_dg.csv_parser(df_raw, CSV_PARSER)

# Calculate some fields.
df['name'] = df['name'].str.title()
df['addr:province'] = df['addr:province'].str.title()
df['addr:city'] = df['addr:city'].str.title()


# Create a source column with "Opendata Generalitat Valenciana"
df['source'] = "Opendata Generalitat Valenciana"
df['amenity'] = 'pharmacy'


df.head(10)

,source:pkey,addr:province,addr:city,name,addr:full,source,amenity
0,A-164-F,Alicante,Agost,Carlos Gisbert Arques,"PLAZA DE ESPAÑA, 19, 03698",Opendata Generalitat Valenciana,pharmacy
1,A-285-F,Alicante,Agost,Francisco Ayuso Macia,"AVDA Virgen de la Paz, 30, 03698",Opendata Generalitat Valenciana,pharmacy
2,A-510-F,Alicante,Agres,Inmaculada Ferrero Perez,"CALLE San Antonio, 13, 03837",Opendata Generalitat Valenciana,pharmacy
3,A-540-F,Alicante,Aigues,Javier Villamayor Piñas,"CALLE CANALEJAS, 12, 03569",Opendata Generalitat Valenciana,pharmacy
4,A-13-F,Alicante,Albatera,Isabel Balsameda Morales,"CALLE Ramon y Cajal, 6, 03340",Opendata Generalitat Valenciana,pharmacy
5,A-14-F,Alicante,Albatera,"Maria Jose Balmaseda Del Alamo, Arturo Corbach...","CTRA HONDON. EDIF.ROMEO, S/N, 03340",Opendata Generalitat Valenciana,pharmacy
6,A-580-F,Alicante,Albatera,Maria Del Mar Garcia Molina,"AVDA Calvario, 39, 03340",Opendata Generalitat Valenciana,pharmacy
7,A-814-F,Alicante,Albatera,Francisco Garcia Canovas,"CALLE Meson, 45, 03340",Opendata Generalitat Valenciana,pharmacy
8,A-531-F,Alicante,Alcalali,Encarnacion Moll Mengual,"C Calvari, 7, 03728",Opendata Generalitat Valenciana,pharmacy
9,A-748-F,Alicante,Alcalali,Raquel Ferrer Gonzalez,"CALLE SALAMANCA, 19, 03723",Opendata Generalitat Valenciana,pharmacy


## Geocode dataframe

In [13]:
# Geocode
from geopy.geocoders import Nominatim
geolocator = Nominatim(timeout=10, user_agent = "myGeolocator")

df = df.iloc[0:25, :]
df['addr_full'] = df['addr:full'] + ', ' + df['addr:city']
df['gcode'] = df.addr_full.apply(geolocator.geocode)

# Store rows that have not been geolocated.
df_not_found = df[df['gcode'].isnull()]

df_not_found


# Proceed with geolocated values only.
df_loc = df[df['gcode'].notna()]

# Generate a `lat` and `lon` columns with latitude and longitude values.
df_loc['lat'] = [g.latitude for g in df_loc.gcode]
df_loc['lon'] = [g.longitude for g in df_loc.gcode]

df_loc

,source:pkey,addr:province,addr:city,name,addr:full,source,amenity,addr_full,gcode,lat,lon
2,A-510-F,Alicante,Agres,Inmaculada Ferrero Perez,"CALLE San Antonio, 13, 03837",Opendata Generalitat Valenciana,pharmacy,"CALLE San Antonio, 13, 03837, Agres","(Farmacia Lda Inmaculada Ferrero, 13, Carrer S...",38.780493,-0.516679
4,A-13-F,Alicante,Albatera,Isabel Balsameda Morales,"CALLE Ramon y Cajal, 6, 03340",Opendata Generalitat Valenciana,pharmacy,"CALLE Ramon y Cajal, 6, 03340, Albatera","(6, Calle Ramón y Cajal, Albatera, el Baix Seg...",38.178251,-0.869586
7,A-814-F,Alicante,Albatera,Francisco Garcia Canovas,"CALLE Meson, 45, 03340",Opendata Generalitat Valenciana,pharmacy,"CALLE Meson, 45, 03340, Albatera","(Farmacia Santa Rita, 45, Calle Mesón, Albater...",38.178061,-0.871879
8,A-531-F,Alicante,Alcalali,Encarnacion Moll Mengual,"C Calvari, 7, 03728",Opendata Generalitat Valenciana,pharmacy,"C Calvari, 7, 03728, Alcalali","(7, Carrer Calvari, Alcalalí, la Marina Alta, ...",38.749951,-0.039835
9,A-748-F,Alicante,Alcalali,Raquel Ferrer Gonzalez,"CALLE SALAMANCA, 19, 03723",Opendata Generalitat Valenciana,pharmacy,"CALLE SALAMANCA, 19, 03723, Alcalali","(19, Calle Salamanca, la Llosa de Camacho, Alc...",38.771219,-0.009237
12,A-104-F,Alicante,Alcoy/Alcoi,Maria Vicenta Cerda Ferre,"C/ Santa Rosa, 19, 03802",Opendata Generalitat Valenciana,pharmacy,"C/ Santa Rosa, 19, 03802, Alcoy/Alcoi","(Carrer Santa Rosa, Santa Rosa, Centre, Alcoi ...",38.697752,-0.483977
14,A-126-F,Alicante,Alcoy/Alcoi,Santiago Llull Mir,"C/ Ibi, 20, 03802",Opendata Generalitat Valenciana,pharmacy,"C/ Ibi, 20, 03802, Alcoy/Alcoi","(20, Carrer Ibi, Santa Rosa, Centre, Alcoi / A...",38.696346,-0.485637
15,A-171-F,Alicante,Alcoy/Alcoi,Francisco Enrique Dominguez,"C/ Oliver, 15, 03802",Opendata Generalitat Valenciana,pharmacy,"C/ Oliver, 15, 03802, Alcoy/Alcoi","(15, Carrer Oliver, Santa Rosa, Centre, Alcoi ...",38.694502,-0.482166
16,A-184-F,Alicante,Alcoy/Alcoi,María Teresa Penades Pinilla,"C/ Sant Nicolau, 18, 03801",Opendata Generalitat Valenciana,pharmacy,"C/ Sant Nicolau, 18, 03801, Alcoy/Alcoi","(Farmàcia Sant Nicolau, 18, Carrer Sant Nicola...",38.697294,-0.473103
17,A-226-F,Alicante,Alcoy/Alcoi,Margarita Robles Saez,"C/ Sant Llorenç, 25, 03801",Opendata Generalitat Valenciana,pharmacy,"C/ Sant Llorenç, 25, 03801, Alcoy/Alcoi","(25, Carrer Sant Llorenç, Viaducte, Centre, Al...",38.697356,-0.474347


Now display all the rows that haven't been geolocated.

In [14]:
df_not_found

,source:pkey,addr:province,addr:city,name,addr:full,source,amenity,addr_full,gcode
0,A-164-F,Alicante,Agost,Carlos Gisbert Arques,"PLAZA DE ESPAÑA, 19, 03698",Opendata Generalitat Valenciana,pharmacy,"PLAZA DE ESPAÑA, 19, 03698, Agost",None
1,A-285-F,Alicante,Agost,Francisco Ayuso Macia,"AVDA Virgen de la Paz, 30, 03698",Opendata Generalitat Valenciana,pharmacy,"AVDA Virgen de la Paz, 30, 03698, Agost",None
3,A-540-F,Alicante,Aigues,Javier Villamayor Piñas,"CALLE CANALEJAS, 12, 03569",Opendata Generalitat Valenciana,pharmacy,"CALLE CANALEJAS, 12, 03569, Aigues",None
5,A-14-F,Alicante,Albatera,"Maria Jose Balmaseda Del Alamo, Arturo Corbach...","CTRA HONDON. EDIF.ROMEO, S/N, 03340",Opendata Generalitat Valenciana,pharmacy,"CTRA HONDON. EDIF.ROMEO, S/N, 03340, Albatera",None
6,A-580-F,Alicante,Albatera,Maria Del Mar Garcia Molina,"AVDA Calvario, 39, 03340",Opendata Generalitat Valenciana,pharmacy,"AVDA Calvario, 39, 03340, Albatera",None
10,A-642-F,Alicante,Alcocer De Planes,Susana Martinez Valles,"CALLE San Jose, 1, 03841",Opendata Generalitat Valenciana,pharmacy,"CALLE San Jose, 1, 03841, Alcocer De Planes",None
11,A-546-F,Alicante,Alcoleja,Gemma Bernabeu Ivorra,"AVDA DEL PAIS VALENCIANO, 10, 03814",Opendata Generalitat Valenciana,pharmacy,"AVDA DEL PAIS VALENCIANO, 10, 03814, Alcoleja",None
13,A-122-F,Alicante,Alcoy/Alcoi,"Inés Llopis Boluda, Maria José Llopis Boluda","C/ Na Saurina d, EntenÃ§a, 45, 03803",Opendata Generalitat Valenciana,pharmacy,"C/ Na Saurina d, EntenÃ§a, 45, 03803, Alcoy/Alcoi",None
18,A-227-F,Alicante,Alcoy/Alcoi,Isabel Maria Ramirez Rodriguez,"C/ AlÃ§amora (els), 19, 03802",Opendata Generalitat Valenciana,pharmacy,"C/ AlÃ§amora (els), 19, 03802, Alcoy/Alcoi",None
19,A-229-F,Alicante,Alcoy/Alcoi,Miguel Domenech Lloret,"AV Alameda de Camilo Sesto, 53, 03803",Opendata Generalitat Valenciana,pharmacy,"AV Alameda de Camilo Sesto, 53, 03803, Alcoy/A...",None


## Export clean data

If the attributes above are correct, we have to proceed to export them into a `CSV` and `geojson` files that can be used in the Task Manager's project.

In [15]:
# Generate  a CSV File.
df_loc.to_csv('data/processed/pharmacies_cval.csv', index = False)

In [16]:
# Convert dataframe into a GeoDataframe.
gdf = gpd.GeoDataFrame(
    df_loc,
    geometry=gpd.points_from_xy(df_loc.lon, df_loc.lat))


gdf
# Export to geojson.
df_loc.to_file('data/processed/pharmacies_cval.geojson', driver='GeoJSON')

AttributeError: 'DataFrame' object has no attribute 'to_file'

The resulting geojson file can be found in the folder [`/data/processed/bcn_trees.geojson`](https://github.com/mapcolabora/osm_imports_preparations/blob/master/imports/bcn_trees/data/processed/bcn_trees.geojson) in this repo.

**TODOs:**

* Drop latitude and longitude fields from GeoJson, [Issue #19](https://github.com/mapcolabora/osm_imports_preparations/issues/19) (help appreciated!)
* Generate a working geojson (otherwise, a CSV can be generated)
